In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

Instructions for updating:
Colocations handled automatically by placer.


W0614 20:43:26.954489 140590461744960 deprecation.py:323] From /home/adrian/anaconda3/envs/image-captioning/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
data = pd.read_csv('./dataset/spam.csv', encoding='latin-1')

In [4]:
y = list(data['v1'])
x = list(data['v2'])

In [5]:
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [6]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

In [7]:
test = encode(le, ['ham', 'spam', 'ham', 'ham'])
test

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]], dtype=float32)

In [8]:
untest = decode(le, test)
untest

array(['ham', 'spam', 'ham', 'ham'],
      dtype='<U4')

In [9]:
x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])

In [11]:
x_train[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [13]:
y_train[0]

array([ 1.,  0.], dtype=float32)

In [14]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [15]:
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [16]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0614 20:44:23.406486 140590461744960 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [25]:
def embedding(x):
    em = embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


with tf.Session() as session:
    train = np.asarray(x_enc[:1])
    r = session.run(embedding(train))
    print(r)

TypeError: Can't convert 'text': Shape TensorShape([]) is incompatible with TensorShape([Dimension(None)])

In [16]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./elmo-model.h5')


Instructions for updating:
Use tf.cast instead.


W0614 20:21:25.770237 140223534167872 deprecation.py:323] From /home/adrian/anaconda3/envs/image-captioning/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/1
5000/5000 [==============================] - 52s 10ms/step - loss: 0.0965 - acc: 0.9650


In [17]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=32)

In [18]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [19]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

[[495   3]
 [  6  68]]
              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       498
        spam       0.96      0.92      0.94        74

    accuracy                           0.98       572
   macro avg       0.97      0.96      0.96       572
weighted avg       0.98      0.98      0.98       572

